In [3]:
# based on exmaples from
# https://planetarycomputer.microsoft.com/docs/tutorials/cloudless-mosaic-sentinel2/
# https://planetarycomputer.microsoft.com/dataset/sentinel-2-l2a#Example-Notebook
from pystac.extensions.eo import EOExtension as eo
import pystac_client
import planetary_computer
import glob
import rioxarray as rxr
from rioxarray.merge import merge_arrays
import re, os
import datetime
import pandas as pd
from shapely.geometry import box
import odc.stac
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import rasterio as rio

In [4]:
def cop30_for_aso(aso_raster_fn, dataset_path):
    aso_raster = rxr.open_rasterio(aso_raster_fn).squeeze()
    aso_raster = aso_raster.where(aso_raster>=0, drop=True)
    aso_raster = aso_raster.interpolate_na(dim='x')
    bounds_latlon = box(*aso_raster.rio.transform_bounds("EPSG:4326"))
    
    catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1")

    search = catalog.search(
        collections=["cop-dem-glo-30"],
        intersects=bounds_latlon)

    # Check how many items were returned
    items = search.item_collection()
    print(f"Returned {len(items)} Items")
    
    data = []
    for item in items:
        dem_path = planetary_computer.sign(item.assets['data']).href
        data.append(rxr.open_rasterio(dem_path))
    cop30_da = merge_arrays(data)  
    
    # clip to ASO extent
    cop30_da = cop30_da.squeeze().rio.reproject_match(aso_raster, resampling=rio.enums.Resampling.bilinear)
    os.makedirs(os.path.join(dataset_path, 'cop30'), exist_ok=True)
    cop30_da.to_netcdf(os.path.join(dataset_path, 'cop30', f'cop30_for_{aso_raster_fn.replace('\\', '/').split("/")[-1][:-4]}.nc'))
    
    #return cop30_da, aso_raster

In [3]:
#fn = '/home/jovyan/deep-snow/data/ASO/ASO_50m_SD_cleaned/utm11n/ASO_50M_SD_Carson_20220311_clean.tif'

In [4]:
#cop30_da, aso = cop30_for_aso(fn)

In [5]:
# f, ax = plt.subplots(1, 2, figsize = (10, 10))
# ax[0].imshow(cop30_da)
# ax[0].set_aspect('equal')
# ax[1].imshow(aso, vmin=0, vmax=2, cmap = "Blues")
# ax[1].set_aspect('equal')
# plt.tight_layout()

In [5]:
def cop30_for_aso_all(dir_path, dataset_path):
    error_paths = []
    raster_paths = glob.glob(f'{dir_path}/*/ASO_50M_SD*.tif')[:1]
    for i, path in enumerate(raster_paths):
        print(f'----\nworking on {path.replace('\\', '/').split("/")[-1]}, {i+1}/{len(raster_paths)}\n----')
        
        try:
            cop30_for_aso(path, dataset_path)
        except Exception as e:
            print(e)
            print('encountered error downloading, skipping for now')
            error_paths.append(path)

In [6]:
dir_path = r"/home/ayushg12/ML_GEO2024_ayushg12/mlgeo-2024-deep-snow/final_data/ASO_50m_SD_cleaned"
dataset_path = r"/home/ayushg12/ML_GEO2024_ayushg12/mlgeo-2024-deep-snow/final_data"
cop30_for_aso_all(dir_path, dataset_path)

----
working on ASO_50M_SD_USCALB_20170815_clean.tif, 1/1
----
Returned 2 Items


In [7]:
import time
import os

# Function to perform STAC search with retries
def search_with_retries(catalog, bounds_latlon, max_retries=5, delay=5):
    attempt = 0
    while attempt < max_retries:
        try:
            search = catalog.search(
                collections=["cop-dem-glo-30"],
                intersects=bounds_latlon
            )
            return search.item_collection()
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            attempt += 1
            time.sleep(delay)
    raise Exception(f"Search failed after {max_retries} retries.")

# Function to process the ASO raster and fetch COP30 data
def cop30_for_aso(aso_raster_fn, dataset_path):
    # Load and preprocess the ASO raster
    aso_raster = rxr.open_rasterio(aso_raster_fn).squeeze()
    aso_raster = aso_raster.where(aso_raster >= 0, drop=True)
    aso_raster = aso_raster.interpolate_na(dim='x')
    bounds_latlon = box(*aso_raster.rio.transform_bounds("EPSG:4326"))
    # Open STAC catalog
    catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
    # Perform the search with retries
    try:
        items = search_with_retries(catalog, bounds_latlon)
        print(f"Returned {len(items)} Items")
    except Exception as e:
        print(f"Error during STAC search: {e}")
        return
    # If items were found, process them
    data = []
    for item in items:
        dem_path = planetary_computer.sign(item.assets['data']).href
        data.append(rxr.open_rasterio(dem_path))
    # Merge and reproject COP30 data to match ASO raster
    cop30_da = merge_arrays(data)
    cop30_da = cop30_da.squeeze().rio.reproject_match(aso_raster, resampling=rio.enums.Resampling.bilinear)
    # Save the result as NetCDF
    # Extract the base filename from aso_raster_fn
    base_filename = os.path.splitext(os.path.basename(aso_raster_fn))[0]
    # Save the result as NetCDF
    output_dir = os.path.join(dataset_path, 'cop30')
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, f"cop30_for_{base_filename}.nc")
    cop30_da.to_netcdf(output_path)
    #print(f"Saved NetCDF to {output_path}")

# Function to process all ASO rasters in a directory
def cop30_for_aso_all(dir_path, dataset_path):
    error_paths = []
    raster_paths = glob.glob(os.path.join(dir_path, '*', 'ASO_50M_SD*.tif'))
    for i, path in enumerate(raster_paths):
        print(f'----\nWorking on {path.replace('\\', '/').split("/")[-1]}, {i+1}/{len(raster_paths)}\n----')

        try:
            cop30_for_aso(path, dataset_path)
        except Exception as e:
            print("\n")
            print(e, path)
            print("\n")
            print('Encountered an error, skipping for now')
            error_paths.append(path)

In [8]:
aso_raster = rxr.open_rasterio(r"/home/ayushg12/ML_GEO2024_ayushg12/mlgeo-2024-deep-snow/final_data/ASO_50m_SD_cleaned/utm10n/ASO_50M_SD_American_20230131_clean.tif").squeeze()
aso_raster = aso_raster.where(aso_raster >= 0, drop=True)
aso_raster = aso_raster.interpolate_na(dim='x')
bounds_latlon = box(*aso_raster.rio.transform_bounds("EPSG:4326"))
# Open STAC catalog
catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
# Perform the search with retries
try:
    items = search_with_retries(catalog, bounds_latlon)
    print(f"Returned {len(items)} Items")
except Exception as e:
    print(f"Error during STAC search: {e}")
# If items were found, process them
data = []
for item in items:
    dem_path = planetary_computer.sign(item.assets['data']).href
    data.append(rxr.open_rasterio(dem_path))
# Merge and reproject COP30 data to match ASO raster
cop30_da = merge_arrays(data)
cop30_da = cop30_da.squeeze().rio.reproject_match(aso_raster, resampling=rio.enums.Resampling.bilinear)

Returned 4 Items


In [ ]:
cop30_da.to_netcdf(rf"/home/ayushg12/ML_GEO2024_ayushg12/mlgeo-2024-deep-snow/final_data/cop30/test.nc")

: 

In [ ]:
dir_path = r"/home/ayushg12/ML_GEO2024_ayushg12/mlgeo-2024-deep-snow/final_data/ASO_50m_SD_cleaned"
dataset_path = r"/home/ayushg12/ML_GEO2024_ayushg12/mlgeo-2024-deep-snow/final_data"
cop30_for_aso_all(dir_path, dataset_path)

----
Working on ASO_50M_SD_USCALB_20170815_clean.tif, 1/252
----
Returned 2 Items
----
Working on ASO_50M_SD_Carson_20220311_clean.tif, 2/252
----
Returned 4 Items
----
Working on ASO_50M_SD_USCAMB_20190703_clean.tif, 3/252
----
Returned 1 Items
----
Working on ASO_50M_SD_Merced_20230515_clean.tif, 4/252
----
Returned 1 Items
----
Working on ASO_50M_SD_USCARC_20170717_clean.tif, 5/252
----
Returned 1 Items
----
Working on ASO_50M_SD_Merced_20220513_clean.tif, 6/252
----
Returned 1 Items
----
Working on ASO_50M_SD_USCATB_20160426_clean.tif, 7/252
----
Returned 2 Items
----
Working on ASO_50M_SD_USCAKN_20180426_clean.tif, 8/252
----
Returned 4 Items
----
Working on ASO_50M_SD_Tuolumne_20230316_clean.tif, 9/252
----
Returned 4 Items
----
Working on ASO_50M_SD_USCAMB_20190329_clean.tif, 10/252
----
Returned 1 Items
----
Working on ASO_50M_SD_Carson_20230607_clean.tif, 11/252
----
Returned 4 Items
----
Working on ASO_50M_SD_USCAJW_20190605_clean.tif, 12/252
----
Returned 4 Items
----
Workin